In [ ]:
import numpy as np

En esta práctica veremos varios ejemplos de exploraciones exhaustivas, es decir, ir encontrando en el espacio de búsqueda elementos que cumplen ciertas características deseadas, una forma común de hacer esto es utilizando la técnica de backtracking (o vuelta hacia atrás), en donde se van buscando elementos con determinadas características en cada paso.

# Ejemplos

## Cambio
Recordando un poco el ejemplo que vimos anteriormente sobre monedas y sus denominaciones, consideremos una lista $L$ de denominaciones posibles para nuestras monedas, y nuestro problema ahora será determinar de cuántas formas se puede formar el número $n$ utilizando dichas denominaciones (suponiendo que tenemos tantas monedas de cada denominación como sea necesario).

Dada la lista $L$, podemos hacer backtracking del siguiente modo:

* Iteramos sobre los elementos de la lista, y definimos una función `count(i,k)` que nos dice de cuántas formas podemos sumar a $k$ con denominaciones $L[i], L[i+1], \dots$.
* Podemos calcular el valor de `count(i,k)` de manera recursiva del siguiente modo:
    * Si $L[i] > k$, esto nos dice que la moneda actual (`L[i]`) es más grande que el dinero que queremos partir (`k`), así que nos movemos a la siguiente, i.e. `count(i,k) = count(i+1, k)`.
    * Si $L[i] \leq k$, entonces podemos cambiar el dinero con la moneda actual. Entonces, consideramos dos casos: uno en el que sí lo cambiamos y nos quedamos en la misma moneda (`count(i, k-L[i])`) y otro en el que no lo hacemos y simplemente pasamos a la siguiente moneda (`count(i+1, k)`).

Veamos una implementación de este algoritmo:

In [ ]:
combs = []

def count(L, k, i=0, out=None):
    if out is None:
        out = []

    if i >= len(L):
        return 0
    if k == 0:
        combs.append(out)
        return 1
    if L[i] > k:
        return count(L, k, i+1, out)
    else:
        temp = out.copy()
        temp.append(L[i])
        return count(L, k-L[i], i, temp) + count(L, k, i+1, out)

L = [2, 8, 4, 6]
print(count(L, 10))
print(combs)

6
[[2, 2, 2, 2, 2], [2, 2, 2, 4], [2, 2, 6], [2, 8], [2, 4, 4], [4, 6]]


## Reinas

Un famoso problema algorítmico es que dado un tablero de ajedrez (de $8 \times 8$), se determine cuántas formas hay de colocar $8$ reinas en el tablero de tal manera que no haya dos que se ataquen entre sí. Este es un ejemplo clásico de la técnica de backtracking, resolveremos el problema para un tablero de $n\times n$ con $n$ reinas.

Primero, representaremos el tablero como una lista $Q$ de tamaño $n$. La entrada $Q_i$ será un número $0\leq j < n$ que nos dirá la columna en la cual se encuentra la reina de la $i$-ésima fila (de abajo hacia arriba). Por ejemplo, para el siguiente arreglo de reinas:

![chess](https://github.com/RodolfoFigueroa/madi2022-1/blob/main/Unidad_3/chess.png?raw=1)

Su representación sería `[1, 3, 0, 2]`.

Luego, definimos la siguiente función:

In [ ]:
def place_queens(n, row=0, Q=None, queens=None):
    if Q is None:
        Q = [None] * n
    if queens is None:
        queens = []
        
    if row == n:
        queens.append(Q)
        return
    
    for col in range(n):
        legal = True
        for j in range(row):
            if Q[j] == col or Q[j] == col+row-j or Q[j] == col-row+j:
                legal = False
                break
        if legal:
            Q[row] = col
            place_queens(n, row+1, Q.copy(), queens)
    return queens

Esta acepta cuatro argumentos:
* `n`: Tamaño del tablero
* `row`: Fila a partir de la cual se intenta insertar reinas, después de la cual se sube recursivamente hasta llegar a la última. Por defecto es cero (i.e., llenar el tablero completo)
* `Q`: Lista de posiciones de las reinas, en el formato descrito previamente. Al principio está lleno de `None`.
* `queens`: Lista de todas las soluciones posibles. Al principio es vacía.

El algoritmo consiste en ir insertando reinas recursivamente, empezando de la fila de abajo. Se utiliza un bucle para intentar insertarla en cada columna de la fila actual, checando todas las filas anteriores para ver si la posición es válida. Si sí lo es, se inserta y se pasa a la siguiente.

Lo intentamos con un tablero de $4\times 4$:

In [ ]:
queens = place_queens(4)
queens

[[1, 3, 0, 2], [2, 0, 3, 1]]

## Suma de subconjuntos

Dado un conjunto de números positivos $S$ y un número $x$, queremos determinar si existe un subconjunto $U\subseteq S$ tal que la suma de los elementos de $U$ es $x$.

Primero, notemos que hay dos casos base: si $x$ es cero, regresamos `verdadero` inmediatamente, ya que la suma del conjunto vacío lo cumple. Por otro lado, si $x<0$, o si $x\neq 0$ pero $S$ es vacío, regresamos `falso`, ya que no existe solución.

Por otro lado, para el caso general, consideremos un elemento $s\in S$ arbitrario. Existe un subconjunto de $S$ que suma a $x$ si y solo si alguna de las dos proposiciones siguientes es verdadera:

* Existe un subconjunto $U\subseteq S$ que suma a $x$ y $s\in U$.
* Existe un subconjunto $U\subseteq S$ que suma a $x$ y $s\notin U$.

El primer caso implica que debe de existir un subconjunto de $S$ que no incluya a $s$ y que sume a $x-s$. En el segundo caso, debe de existir un subconjunto de $S$ que no incluya a $s$, y que sume a $x$. Con esto, podemos reducir el problema a los siguientes dos subproblemas:

* Subconjunto de $S\setminus \{s\}$ que sume a $x-s$.
* Subconjunto de $S\setminus \{s\}$ que sume a $x$.

Así, podemos definir el algoritmo recursivo:

In [ ]:
def subset_sum(S, x):
    if x == 0:
        return True
    elif len(S) == 0 or x < 0:
        return False
    
    S_minus = S.copy()
    s = S_minus.pop()
    b1 = subset_sum(S_minus, x-s)
    b2 = subset_sum(S_minus, x)
    
    return b1 or b2

Probándolo en un conjunto que sí sabemos que funciona:

In [ ]:
S = [1,14,8,4]
x = 15
subset_sum(S, x)

True

Y en uno que no funciona:

In [ ]:
S = [1,3,8,4]
x = 2
subset_sum(S, x)

False

Hacer copias de un arreglo es una operación costosa. Es más eficiente pasar el mismo arreglo cada vez, y simplemente cambiar los índices que consideramos:

In [ ]:
def subset_sum_index(S, x, r=None):
    if r is None:
        r = len(S)
        
    if x == 0:
        return True
    elif r == 0 or x < 0:
        return False
    
    s = S[r-1]
    b1 = subset_sum_index(S, x-s, r=r-1)
    b2 = subset_sum_index(S, x, r=r-1)
    
    return b1 or b2

In [ ]:
S = [1,3,8,4]
x = 15
subset_sum_index(S, x)

True

Comparando los tiempos de ejcución para una lista de enteros grande:

In [ ]:
big_list = list(np.random.randint(0, 100, 300))

In [ ]:
%%timeit
#subset_sum(big_list, x)

1 loop, best of 5: 407 ms per loop


In [ ]:
%%timeit
#subset_sum_index(big_list, x)

1 loop, best of 5: 361 ms per loop


# Ejercicios

## Ejercicio 1
Dado un conjunto de palabras y un string, describe e implementa un algoritmo que permita contar cuántas oraciones diferentes puede formar la string en cuestión suponiendo que las únicas palabras que existen son las del conjunto inicial. Por ejemplo, si el conjunto de palabras es $\{hola, ola, h\}$, la string "holah" tiene dos posibles interpretaciones, una es "hola h", y la segunda "h ola h". Verifica tu algoritmo con: 

*   Conjunto de palabras `{a, as, tin, tinar, san, sana, atina, arce, ce, atinar}`, y la string "atinarcesanas".
*   Conjunto de palabras `{i, like, ice, and, cream, icecream, man, go, mango}` y la string "ilikeicecreamandmango".

*descripción ejercicio1*

Primero pedimos la palabra desada y el conjunto de palabras por buscar,así mismo asignamos j,i=o y otras dos variables como strings vacios conj y pal.

Proseguimos a cjecar si j es mayor que la longitud de la palabra y ademas si pal es distinto a al conjunto de palabras, vemos que no hay palabra por poner, así que regresamos un cero.

Si el conjunto de palabras es igual a conj 
Abremos terminado para una iteración y regresamos el string vacio pal,ahora si i es mayor al conjunto de palabras , significa que no encontramos alguna que cumpla las especificaciones y regresamos 0.
Hacemos otro ciclo for en donde si S[j] es igual a uno de las palabras del conjunto de palabras.
Hacemos copia de nuestro string conj y a conj lesumamos la palabra en S[j], otra copia de la iteración i que llevemos y a i le sumamos la longitud de la palabra encontrada en S[j],por ultimo a pal la igualamos a pal más una separación y la pal encontrada en S[j].
Hacemos backtracking donde regresamos la misma función con los nuevos valores + la función con j=0.
Si no se cumple esto regresamos la función con j+1



In [ ]:
def cant(S, x, i=0, j=0,conj="", pal=""):
  if j>len(S)-1 and conj!=x:
    return 0
  if x==conj:
    print(pal)
    print('Palabra enconontrada')
    return 1
    print('El total de palabras es,',1)
  if(i>len(x)):
    return 0
  if S[j]==x[len(conj):len(conj)+len(S[j])]:
    pal2=pal
    ii=i
    i+=len(S[j])
    tempS=conj
    conj=conj+S[j]
    pal=pal+"_"+S[j]
    return cant(S, x, ii,j+1, tempS, pal2)+ cant(S, x, i,0,conj,pal)
  else:
    return cant(S, st, i,j+1, conj, pal)

Ejemplo 1

In [ ]:
S=["a", "as", "tin", "tinar", "san", "sana", "atina", "arce", "ce", "atinar"]
st="atinarcesanas"
print(cant(S, st))



_atinar_ce_san_as
Palabra enconontrada
_a_tinar_ce_san_as
Palabra enconontrada
_a_tin_arce_san_as
Palabra enconontrada
3


Ejemplo 2

In [ ]:
S=["i", "like", "ice", "and", "cream", "icecream", "man", "go", "mango"]
st="ilikeicecreamandmango"
print(cant(S, st))

_i_like_icecream_and_mango
Palabra enconontrada
_i_like_icecream_and_man_go
Palabra enconontrada
_i_like_ice_cream_and_mango
Palabra enconontrada
_i_like_ice_cream_and_man_go
Palabra enconontrada
4


## Ejercicio 2
Supón que ahora en el ejemplo 1 no se tienen tantas monedas como se deseen. Es decir, se tiene una lista $L$ de denominaciones posibles, y un entero $k$ que nos indica que tenemos exactamente $k$ monedas de cada denominación posible. Describe e implementa un algoritmo que permita contar de cuántas formas se puede formar un entero $n$ con monedas de las denominaciones dadas, y usando a lo más $k$ monedas de cada denominación.

*Descripción Ejercicio 2.* 
Priero pedimos la lista L de denominaciones, el número al que se quiere llegar y la lista de cuantas monedas tenemos de cada denominación.
Se genera una lista , ahora si el valor de la i-esima moneda es mayor que k , se llama a la misma función de manera recursiva solo que ahora la sumamos uno a nuestro valor i, ahora si las monedas que tenemos (lista2) es menor o igual a cero volvemos a llamar de manera recursiva a la fuinción pero con i+1, de otra forma lo hacemos una copia de nuestra lista  (out)a la cual le adjuntamos el i-esimo valor de la primera lista y le restamos 1 al i-esimo valor de la segunda lista, esto es ya que tenemos una moneda menos de tal denominación y hacemos una copia de la lista 2 ya modificada y llamamos de manera recursiva a la función pero ahora el valor buscado ser'a k-el valor de L[i] y las cantidad de monedas que tenemos será la lista dos ya modificada y a esta le sumamos de nuevo a la función  pero con i+1

In [ ]:
combs = []
def monlim(L, k, lista2, i=0, out=None):
    if out is None:
        out = []
    if i >= len(L):
        return 0
    if k == 0:
        combs.append(out)
        return 1
    if L[i] > k:
        return monlim(L, k, lista2, i+1,out)
    elif(lista2[i]<=0):
        return(monlim(L, k, lista2, i+1,out))
    else:
        temp = out.copy()
        temp.append(L[i])
        lista2[i]-=1
        tempL=lista2.copy()
        return monlim(L, k-L[i], tempL,i, temp) + monlim(L, k, lista2, i+1, out)


Algún ejemplo de el algoritmo

In [ ]:
L = [2, 8, 4, 6]
L2=[1,1,1,1]
print(monlim(L, 12, L2))
print(combs)

2
[[2, 4, 6], [8, 4]]
